3.2 关于多轮对话与上下文记忆

In [ ]:
import os
import dotenv
from langchain_openai import ChatOpenAI
dotenv.load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
# os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")
chat_model = ChatOpenAI(
    model="gpt-4o-mini",
    max_tokens = 1024
)


大模型没有上下文记忆能力

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage
sys_message = SystemMessage(
    content="我是一个人工智能的助手，我的名字叫小智",
)
human_message = HumanMessage(content="猫王是一只猫吗？")
messages = [sys_message, human_message]
#调用大模型，传入messages
response = chat_model.invoke(messages)
print(response.content)

response1 = chat_model.invoke("你叫什么名字？")
print(response1.content)
# 不会记得自己叫小智

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage
sys_message = SystemMessage(
content="我是一个人工智能的助手，我的名字叫小智",
)

human_message = HumanMessage(content="猫王是一只猫吗？")
human_message1 = HumanMessage(content="你叫什么名字？")
# ai只会回答最后一个问题
# 前面一大窜都是作为过程存在，ai只关心最后一个HumanMessage
messages = [sys_message, human_message,human_message1]
#调用大模型，传入messages
response = chat_model.invoke(messages)
print(response.content)

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage

sys_message = SystemMessage(
    content="我是一个人工智能的助手, 我的名字叫小智",
)
human_message = HumanMessage(content="猫是一只猫吗？")

sys_message1 = SystemMessage(
    content="我可以做很多事情, 有需要就找我吧",
)

human_message1 = HumanMessage(content="你叫什么名字？")

messages = [sys_message, human_message, sys_message1, human_message1]

# 调用大模型, 传入messages
response = chat_model.invoke(messages)
print(response.content)
# 回复：我叫小智。很高兴认识你！有什么我可以帮助你的吗？

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage
# 第1组
sys_message = SystemMessage(
    content="我是一个人工智能的助手，我的名字叫小智"
)
human_message = HumanMessage(content="猫王是一只猫吗？")
messages = [sys_message, human_message]
# 第2组
sys_message1 = SystemMessage(
content="我可以做很多事情，有需要就找我吧",
)
human_message1 = HumanMessage(content="你叫什么名字？")
messages1 = [sys_message1,human_message1]
#调用大模型，传入messages
response = chat_model.invoke(messages)
print(response.content)
response = chat_model.invoke(messages1)
print(response.content)
# 说不出来

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

messages = [
    SystemMessage(content="我是一个人工智能助手, 我的名字叫小智"),
    HumanMessage(content="人工智能英语怎么说？"),
    AIMessage(content="AI"),
    HumanMessage(content="你叫什么名字"),
]

messages1 = [
    SystemMessage(content="我是一个人工智能助手, 我的名字叫小智"),
    HumanMessage(content="很高兴认识你"),
    AIMessage(content="我也很高兴认识你"),
    HumanMessage(content="你叫什么名字？"),
]

messages2 = [
    SystemMessage(content="我是一个人工智能助手, 我的名字叫小智"),
    HumanMessage(content="人工智能英语怎么说？"),
    AIMessage(content="AI"),
    HumanMessage(content="你叫什么名字？"),
]

response = chat_model.invoke(messages2)

3. 关于模型调用的方法的说明
invoke() / stream()
batch(): 批量的调用 (Batch invocation)
ainvoke() / astream() / abatch(): 异步方法的调用 (Asynchronous method invocation)

举例1: 阻塞时

In [ ]:
import os
import dotenv
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
dotenv.load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY1")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")#初始化大模型
chat_model = ChatOpenAI(model="gpt-4o-mini")
# 创建消息
messages = [HumanMessage(content="你好，请介绍一下自己")]
# 非流式调用LLM获取响应
response = chat_model.invoke(messages)
# 打印响应内容
print(response)

举例2：流式调用

In [ ]:
import os
# 从 langchain_core.messages 导入 HumanMessage
from langchain_core.messages import HumanMessage
# 从 langchain_openai 导入 ChatOpenAI
from langchain_openai import ChatOpenAI
# 导入 dotenv 库来加载 .env 文件中的环境变量
import dotenv

# 加载环境变量 (例如从 .env 文件中加载 OPENAI_API_KEY 和 OPENAI_BASE_URL)
dotenv.load_dotenv()

# 设置环境变量，确保模型能够找到 API Key 和 Base URL
# 这里的代码假设您在 .env 文件中设置了这些变量
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
# 如果您使用的是非官方的 API 代理，才需要设置 BASE_URL
# os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")

# 初始化大模型
# streaming=True 表示启用流式输出，这是 stream() 方法的关键
chat_model = ChatOpenAI(
    model="gpt-4o-mini",
    streaming=True  # 启用流式输出
)

# 创建消息列表
messages = [HumanMessage(content="你好，请介绍一下自己")]

# 调用 LLM 并流式响应
print("\n[开始流式输出]:")

# 循环处理 chat_model.stream(messages) 返回的每个块 (chunk)
# stream() 方法会生成响应片段
for chunk in chat_model.stream(messages):
    # 逐个打印内容片段
    # end="" 确保每次打印后不换行
    # flush=True 强制刷新输出缓冲区，确保内容立即显示，实现实时流式效果
    print(chunk.content, end="", flush=True)

# 打印换行符以结束输出
print("\n[流式输出结束]")

举例3：使用batch测试批量调用

In [ ]:
import os
import dotenv
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
dotenv.load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
# os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")
# 初始化大模型
chat_model = ChatOpenAI(model="gpt-4o-mini")
messages1 = [SystemMessage(content="你是一位乐于助人的智能小助手"),
HumanMessage(content="请帮我介绍一下什么是机器学习"), ]
messages2 = [SystemMessage(content="你是一位乐于助人的智能小助手"),
HumanMessage(content="请帮我介绍一下什么是AIGC"), ]
messages3 = [SystemMessage(content="你是一位乐于助人的智能小助手"),
HumanMessage(content="请帮我介绍一下什么是大模型技术"), ]
messages = [messages1, messages2, messages3]
# 调用batch
response = chat_model.batch(messages)
print(response)


举例4：同步和异步方法的调用

体会一：

In [ ]:
import time
def call_model():
# 模拟同步API调用
    print("开始调用模型...")
    time.sleep(5) # 模拟调用等待,单位：秒
    print("模型调用完成。")
def perform_other_tasks():
    # 模拟执行其他任务
    for i in range(5):
        print(f"执行其他任务 {i + 1}")
        time.sleep(1) # 单位：秒
def main():
    start_time = time.time()
    call_model()
    perform_other_tasks()
    end_time = time.time()
    total_time = end_time - start_time
    return f"总共耗时：{total_time}秒"
# 运行同步任务并打印完成时间
main_time = main()
print(main_time)

异步调用
异步调用，允许程序在等待某些操作完成时继续执行其他任务，而不是阻塞等待。这在处理I/O操作（如
网络请求、文件读写等）时特别有用，可以显著提高程序的效率和响应性。
举例：
写法1：此写法适合Jupyter Notebook

In [ ]:
import asyncio
import time
async def async_call(llm):
    await asyncio.sleep(5) # 模拟异步操作
    print("异步调用完成")
async def perform_other_tasks():
    await asyncio.sleep(5) # 模拟异步操作
    print("其他任务完成")
async def run_async_tasks():
    start_time = time.time()
    await asyncio.gather(
        async_call(None), # 示例调用，使用None模拟LLM对象
        perform_other_tasks()
    )
    end_time = time.time()
    return f"总共耗时：{end_time - start_time}秒"
# # 正确运行异步任务的方式
# if __name__ == "__main__":
# # 使用 asyncio.run() 来启动异步程序
# result = asyncio.run(run_async_tasks())
# print(result)
# 在 Jupyter 单元格中直接调用
result = await run_async_tasks()
print(result)